# Testing lid driven cavity example

In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, haswell)


In [2]:
] status

Status `~/.julia/environments/v1.5/Project.toml`
  [c52e3926] Atom v0.12.24
  [6e4b80f9] BenchmarkTools v0.5.0
  [aae7a2af] DiffEqFlux v1.24.0
  [0c46a032] DifferentialEquations v6.15.0
  [587475ba] Flux v0.11.1
  [7073ff75] IJulia v1.22.0
  [033835bb] JLD2 v0.2.4
  [e5e0dc1b] Juno v0.8.4
  [19f0c83c] NavierStokes2D v0.1.0 `~/.julia/dev/NavierStokes2D`
  [9e8cae18] Oceananigans v0.44.0
  [14b8a8f1] PkgTemplates v0.7.13
  [91a5bcdd] Plots v1.9.0
  [90137ffa] StaticArrays v0.12.4


In [3]:
using Oceananigans
using Oceananigans: Face, Cell
using Oceananigans.Diagnostics
using Oceananigans.OutputWriters
using Oceananigans.AbstractOperations

In [4]:
# Parameters
Re = 100.0
N = 64
end_time = 10.0

10.0

In [5]:
topology = (Flat, Bounded, Bounded)
domain = (x=(0, 1), y=(0, 1), z=(0, 1))
grid = RegularCartesianGrid(topology=topology, size=(N, N); domain...)

RegularCartesianGrid{Float64, Flat, Bounded, Bounded}
                   domain: x ∈ [0.0, 0.0], y ∈ [0.0, 1.0], z ∈ [0.0, 1.0]
                 topology: (Flat, Bounded, Bounded)
  resolution (Nx, Ny, Nz): (1, 64, 64)
   halo size (Hx, Hy, Hz): (0, 1, 1)
grid spacing (Δx, Δy, Δz): (1.0, 0.015625, 0.015625)

Note: Using the original syntax of

`grid = RegularCartesianGrid(topology=topology, size=(1, N, N); domain...)`

will produce the error

`ArgumentError: length(size) must be 2.`

In [6]:
v_bcs = VVelocityBoundaryConditions(grid,
           top = ValueBoundaryCondition(1.0),
        bottom = ValueBoundaryCondition(0.0))
w_bcs = WVelocityBoundaryConditions(grid,
        north = ValueBoundaryCondition(0.0),
        south = ValueBoundaryCondition(0.0))

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{Nothing,Nothing}
├── y: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Value,Float64}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{NormalFlow,Nothing},BoundaryCondition{NormalFlow,Nothing}}

In [7]:
model = IncompressibleModel(
                       grid = grid,
                   buoyancy = nothing,
                    tracers = nothing,
                   coriolis = nothing,
        boundary_conditions = (v=v_bcs, w=w_bcs),
                    closure = IsotropicDiffusivity(ν=1/Re))

IncompressibleModel{CPU, Float64}(time = 0 seconds, iteration = 0) 
├── grid: RegularCartesianGrid{Float64, Flat, Bounded, Bounded}(Nx=1, Ny=64, Nz=64)
├── tracers: ()
├── closure: IsotropicDiffusivity{Float64,NamedTuple{(),Tuple{}}}
├── buoyancy: Nothing
└── coriolis: Nothing

In [15]:
u, v, w = model.velocities;

In [17]:
max_Δt = 0.25 * model.grid.Δy^2 * Re / 2  # Make sure not to violate diffusive CFL.
wizard = TimeStepWizard(cfl=0.1, Δt=1e-6, max_change=1.1, max_Δt=max_Δt)

cfl = AdvectiveCFL(wizard)
dcfl = DiffusiveCFL(wizard)

CFL{TimeStepWizard{Float64},typeof(Oceananigans.TurbulenceClosures.cell_diffusion_timescale)}(TimeStepWizard{Float64}(0.1, Inf, 1.1, 0.5, 0.0030517578125, 1.0e-6), Oceananigans.TurbulenceClosures.cell_diffusion_timescale)

In [19]:
simulation = Simulation(model, Δt=wizard, stop_time=end_time,
                            iteration_interval=20, parameters=(cfl=cfl, dcfl=dcfl))

Simulation{IncompressibleModel{CPU, Float64}}
├── Model clock: time = 0 seconds, iteration = 0 
├── Next time step (TimeStepWizard{Float64}): 1 μs 
├── Iteration interval: 20
├── Stop criteria: Any[Oceananigans.Simulations.iteration_limit_exceeded, Oceananigans.Simulations.stop_time_exceeded, Oceananigans.Simulations.wall_time_limit_exceeded]
├── Run time: 0 seconds, wall time limit: Inf
├── Stop time: 10 seconds, stop iteration: Inf
├── Diagnostics: OrderedCollections.OrderedDict with no entries
└── Output writers: OrderedCollections.OrderedDict with no entries

In [20]:
run!(simulation)

LoadError: InterruptException: